In [1]:

import sys
import os

In [2]:
print(os.getcwd())


c:\Users\Blen\OneDrive\Desktop\10Academy\Week2\notebooks


In [3]:
current_dir = os.getcwd()

In [4]:
root_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, root_dir)

Importing the Necessary Functionalities from Scripts

In [5]:
from scripts.load_data import load_data_from_postgres, load_data_using_sqlalchemy
from scripts.preprocesser import missing_values_table,handle_missing_identifiers,handle_missing_categorical,handle_missing_numerical,remove_columns,convert_bytes_to_megabytes,remove_outliers,fix_outlier

Loading the Data from Postgres

In [6]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL
df = load_data_from_postgres(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")

c:\Users\Blen\OneDrive\Desktop\10Academy\Week2\scripts\load_data.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Successfully loaded the data


Removing the Unnecessary Columns from our Dataframe

In [7]:
columns_to_remove = ['Start','Start ms','End','End ms','Dur. (ms).1','UL TP < 10 Kbps (%)',
                     '50 Kbps < UL TP < 300 Kbps (%)','UL TP > 300 Kbps (%)','10 Kbps < UL TP < 50 Kbps (%)',
                     'Nb of sec with Vol UL < 1250B','DL TP > 1 Mbps (%)','50 Kbps < DL TP < 250 Kbps (%)',
                     '250 Kbps < DL TP < 1 Mbps (%)','DL TP < 50 Kbps (%)','Nb of sec with 125000B < Vol DL',
                     'Nb of sec with 1250B < Vol UL < 6250B','Nb of sec with 31250B < Vol DL < 125000B','Nb of sec with 37500B < Vol UL',
                     'Nb of sec with 6250B < Vol DL < 31250B','Nb of sec with 6250B < Vol UL < 37500B','Nb of sec with Vol DL < 6250B',
                     'Nb of sec with Vol UL < 1250B','x']

In [8]:
df = remove_columns(df, columns_to_remove)
df.head()

Removed columns: ['Start', 'Start ms', 'End', 'End ms', 'Dur. (ms).1', 'UL TP < 10 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', 'Nb of sec with Vol UL < 1250B', 'DL TP > 1 Mbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP < 50 Kbps (%)', 'Nb of sec with 125000B < Vol DL', 'Nb of sec with 1250B < Vol UL < 6250B', 'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL', 'Nb of sec with 6250B < Vol DL < 31250B', 'Nb of sec with 6250B < Vol UL < 37500B', 'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B']


,Bearer Id,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,5.0,16.0,26.0,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,NaN,6.0,9.0,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,NaN,44.0,44.0,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,NaN,6.0,9.0,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


Handling Missing Values

In [9]:
missing_values = missing_values_table(df)
missing_values

The dataframe has 34columns.
There are 20 columns that have missing values.



,Missing Values,% of Total Values,DType
TCP UL Retrans. Vol (Bytes),96649,64.4,float64
TCP DL Retrans. Vol (Bytes),88146,58.8,float64
HTTP UL (Bytes),81810,54.5,float64
HTTP DL (Bytes),81474,54.3,float64
Avg RTT DL (ms),27829,18.6,float64
Avg RTT UL (ms),27812,18.5,float64
Last Location Name,1153,0.8,object
MSISDN/Number,1066,0.7,float64
Bearer Id,991,0.7,float64
Handset Manufacturer,572,0.4,object


In [10]:
identifier_columns = ['IMEI', 'IMSI', 'MSISDN/Number', 'Bearer Id']
df = handle_missing_identifiers(df, identifier_columns)
df.shape

Dropped 572 rows due to missing values in 'IMEI'
Dropped 0 rows due to missing values in 'IMSI'
Dropped 494 rows due to missing values in 'MSISDN/Number'
Dropped 429 rows due to missing values in 'Bearer Id'


(148506, 34)

In [11]:
df.head()

,Bearer Id,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,5.0,16.0,26.0,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,NaN,6.0,9.0,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,NaN,44.0,44.0,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,NaN,6.0,9.0,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [12]:
columns_for_missing = ['TCP UL Retrans. Vol (Bytes)','TCP DL Retrans. Vol (Bytes)','HTTP UL (Bytes)','HTTP DL (Bytes)',
                       'Avg RTT DL (ms)','Avg RTT UL (ms)']

In [13]:
df = handle_missing_numerical(df, columns_for_missing)
df.head()

c:\Users\Blen\OneDrive\Desktop\10Academy\Week2\scripts\preprocesser.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mean(), inplace=True)


Filled missing values in 'TCP UL Retrans. Vol (Bytes)' using mean
Filled missing values in 'TCP DL Retrans. Vol (Bytes)' using mean
Filled missing values in 'HTTP UL (Bytes)' using mean
Filled missing values in 'HTTP DL (Bytes)' using mean
Filled missing values in 'Avg RTT DL (ms)' using mean
Filled missing values in 'Avg RTT UL (ms)' using mean


,Bearer Id,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.000000,5.000000,23.0,44.0,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.000000,5.000000,16.0,26.0,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,106.199942,17.628232,6.0,9.0,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,106.199942,17.628232,44.0,44.0,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,106.199942,17.628232,6.0,9.0,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [14]:
column = ['Last Location Name']

In [15]:
df = handle_missing_categorical(df, column)
df.head()

Missing values in categorical column 'Last Location Name' filled with 'Unknown'


c:\Users\Blen\OneDrive\Desktop\10Academy\Week2\scripts\preprocesser.py:73: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(fill_value, inplace=True)


,Bearer Id,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.000000,5.000000,23.0,44.0,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.000000,5.000000,16.0,26.0,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,106.199942,17.628232,6.0,9.0,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,106.199942,17.628232,44.0,44.0,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,106.199942,17.628232,6.0,9.0,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [16]:
missing_values = missing_values_table(df)
missing_values

The dataframe has 34columns.
There are 0 columns that have missing values.



,Missing Values,% of Total Values,DType


Converting Bytes to Megabytes

In [17]:
columns_to_convert = ['HTTP UL (Bytes)','HTTP DL (Bytes)','Social Media DL (Bytes)','Social Media UL (Bytes)',
                      'Youtube DL (Bytes)','Youtube UL (Bytes)','Netflix DL (Bytes)','Netflix UL (Bytes)',
                      'Google DL (Bytes)','Google UL (Bytes)','Email DL (Bytes)','Email UL (Bytes)','Gaming DL (Bytes)',
                      'Gaming UL (Bytes)','Other DL (Bytes)','Other UL (Bytes)','Total DL (Bytes)','Total UL (Bytes)']

In [18]:
df = convert_bytes_to_megabytes(df, columns_to_convert)
df.head()

,Bearer Id,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.000000,5.000000,23.0,44.0,...,15.854611,2.501332,8.198936,9.656251,278.082303,14.344150,171.744450,8.814393,36.749741,308.879636
1,1.311448e+19,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.000000,5.000000,16.0,26.0,...,20.247395,19.111729,18.338413,17.227132,608.750074,1.170709,526.904238,15.055145,53.800391,653.384965
2,1.311448e+19,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,106.199942,17.628232,6.0,9.0,...,19.725661,14.699576,17.587794,6.163408,229.584621,0.395630,410.692588,4.215763,27.883638,279.807335
3,1.311448e+19,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,106.199942,17.628232,44.0,44.0,...,21.388122,15.146643,13.994646,1.097942,799.538153,10.849722,749.039933,12.797283,43.324218,846.028530
4,1.311448e+19,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,106.199942,17.628232,6.0,9.0,...,15.259380,18.962873,17.124581,0.415218,527.707248,3.529801,550.709500,13.910322,38.542814,569.138589


Fixing Outliers

In [19]:
columns_to_fix = ['TCP UL Retrans. Vol (Bytes)','TCP DL Retrans. Vol (Bytes)','Dur. (ms)'
                'Avg RTT DL (ms)','Avg RTT UL (ms)','HTTP UL (Bytes)','HTTP DL (Bytes)','Social Media DL (Bytes)','Social Media UL (Bytes)',
                  'Youtube DL (Bytes)','Youtube UL (Bytes)','Netflix DL (Bytes)','Netflix UL (Bytes)',
                  'Google DL (Bytes)','Google UL (Bytes)','Email DL (Bytes)','Email UL (Bytes)','Gaming DL (Bytes)',
                  'Gaming UL (Bytes)','Other DL (Bytes)','Other UL (Bytes)','Total DL (Bytes)','Total UL (Bytes)','Avg Bearer TP DL (kbps)',
                  'Avg Bearer TP UL (kbps)']